# Response Matrix Emulator Example

Assuming you have run most or all of the tutorial notebooks, you hopefully have an appreciation for the computational burden of decision support modeling, especially for management optmization (see Part2_09_mou). This computational burden is obviously much greater for real-world models than it is for the Freyberg domain. Meanwhile, decisionmakers often need results fast, and are often interested in scenarios of perturbations to the original scenario performed. Here enters _emulators_. 

__Emulator__ (aka surrogate or metamodel): a more efficient (e.g., faster running) approximation of a process-based model.

There are many (and more every day) options for groundwater model emulation (see __[Asher et al. (2015)](https://doi.org/10.1002/2015WR016967)__ for a nice review); this notebook walks through how to build and apply a response matrix emulator. 

### Admin

Start off with the usual loading of dependencies and preparing model and PEST files. We will be continuing to work with the modified-Freyberg model (see ["freyberg intro to model"](../part0_02_intro_to_freyberg_model/intro_freyberg_model.ipynb) notebook), and the high-dimensional PEST dataset prepared in the ["freyberg pstfrom pest setup"](../part2_01_pstfrom_pest_setup/freyberg_pstfrom_pest_setup.ipynb)" and ["observation and weights"](../part2_02_obs_and_weights/freyberg_obs_and_weights.ipynb) notebooks. 

For the purposes of this notebook, you do not require familiarity with previous notebooks (but it helps...). 

Simply run the next few cells by pressing `shift+enter`.

In [1]:
import os
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import numpy as np
import pandas as pd
font = {'size'   : 10}
import matplotlib
matplotlib.rc('font', **font)
import matplotlib.pyplot as plt;
import shutil
import psutil

import sys
import pyemu
import flopy
assert "dependencies" in flopy.__file__
assert "dependencies" in pyemu.__file__
sys.path.insert(0,"..")
import herebedragons as hbd
from useful_fxns import *

To maintain continuity in the series of tutorials, we we use the PEST-dataset prepared in the ["observation and weights"](../part2_02_obs_and_weights/freyberg_obs_and_weights.ipynb) tutorial. Run the next cell to copy the necessary files across. Note that you will need to run the previous notebooks in the correct order beforehand.

Specify the path to the PEST dataset template folder. Recall that we will prepare our PEST dataset files in this folder, keeping them separate from the original model files. Then copy across pre-prepared model and PEST files:

### What is a response matrix?

 A matrix that maps changes to decision variables (e.g., model parameters) to changes in outputs (e.g., model observations). This is analogous to a Jacobian matrix underpinning inversion and parameter estimation (__[Doherty, 2015](https://s3.amazonaws.com/docs.pesthomepage.org/documents/all_manuals.zip)__). The response matrix is formulated as follows:


### $ A [outputs\;x\;decision\;variables] = \frac{\partial outputs}{\partial decision variables} \approx \frac{\delta outputs}{\delta decision variables} $

Assuming linearity between decision variables and outputs (which is a big assumption!), the response matrix A is independent of the initial decision variable values. Therefore, the response matrix A can be used to emulate the predicted response (outputs) to a given set of decision variables by multiplying it by a change vector of the decision variables. 

## $ \partial outputs = A * \partial decision\;variables $

Clear as mud? Great, let's put it in practice! We are going to build a response matrix emulator (RME) for none other than the Freyberg model. Let's pretend we are a groundwater aquifer manager tasked with ensuring that water levels in the basin don't fall below some critical threshold given various pumping regimes. The outputs are model heads and the decision variables are pumping rates at each of the wells.

Constructing the response matrix requires running the model once per decision variable (applying the same perturbation or increment to each), and recording all outputs. This gives a matrix of size decision variables by outputs. This can be done using pyemu and PEST++ by using either PESTPP-GLM or PESTPP-OPT. For this example, we will use PESTPP-GLM. 

Let's build this puppy from scratch using `PstFrom`

In [2]:
#create working directory and copy monthly freyberg model files into it
new_d = os.path.join('freyberg_mf6')
og_model_files = os.path.join('..','..','models','monthly_model_files_1lyr_newstress')

if os.path.exists(new_d):
    shutil.rmtree(new_d)

shutil.copytree(og_model_files,new_d)

'freyberg_mf6'

In [3]:
#load the model
sim = flopy.mf6.MFSimulation.load(sim_ws=new_d)
# load flow model
gwf = sim.get_model()
# get spatial reference
modelgrid = gwf.modelgrid
# get zone array
idomain = gwf.dis.idomain.get_data()

loading simulation...
  loading simulation name file...
  loading tdis package...
  loading model gwf6...
    loading package dis...
    loading package ic...
    loading package npf...
    loading package sto...
    loading package oc...
    loading package wel...
    loading package rch...
    loading package ghb...
    loading package sfr...
    loading package obs...
  loading solution package freyberg6...


In [4]:
# run the model 
hbd.prep_bins(new_d)
pyemu.os_utils.run('mf6',cwd=new_d)

                                   MODFLOW 6
                U.S. GEOLOGICAL SURVEY MODULAR HYDROLOGIC MODEL
                   VERSION 6.3.0 release candidate 07/30/2021
                               ***DEVELOP MODE***

   MODFLOW 6 compiled Dec 29 2021 09:30:50 with Intel(R) Fortran Intel(R) 64
   Compiler for applications running on Intel(R) 64, Version 19.1.3.301 Build
                                20200925_000000

This software is preliminary or provisional and is subject to 
revision. It is being provided to meet the need for timely best 
science. The software has not received final approval by the U.S. 
Geological Survey (USGS). No warranty, expressed or implied, is made 
by the USGS or the U.S. Government as to the functionality of the 
software and related material nor shall the fact of release 
constitute any such warranty. The software is provided on the 
condition that neither the USGS nor the U.S. Government shall be held 
liable for any damages resulting from the autho

In [5]:
# start pstfrom
# specify a template directory (i.e. the PstFrom working folder)
template_ws = os.path.join('tmp_pst')
start_datetime= sim.tdis.start_date_time.get_data()
# instantiate PstFrom
pf = pyemu.utils.PstFrom(original_d=new_d, 
                            new_d=template_ws,
                            remove_existing=True, 
                            longnames=True, 
                            spatial_reference=modelgrid, 
                            zero_based=False, 
                            start_datetime=start_datetime, 
                            echo=False)

# add forward run command
pf.mod_sys_cmds.append("mf6") 

#add py pkgs to forward run
pf.extra_py_imports.append("shutil")
pf.extra_py_imports.append("time")
pf.extra_py_imports.append("flopy")
pf.extra_py_imports.append("platform")

In [6]:
# Add well rates as decision variables (1 per well per SP)

files = [f for f in os.listdir(pf.new_d) if f.startswith("freyberg6.wel_stress_period_data_") and f.endswith(".txt")]  
files = sorted(files,
                key=lambda x: int(x.split("data_")[-1].split(".txt")[0]))

for file in files:
    kper = int(file.split(".")[1].split("_")[-1])
    pf.add_parameters(filenames=file,
                        par_style='d',
                        transform="none",
                        index_cols=[0,1], #columns that specify cell location
                        use_cols=[3],       #columns with parameter values
                        par_type="grid",    #each well will be adjustable
                        par_name_base=f"wel_{kper-1}",
                        pargp="welrate", 
                        upper_bound = 0, lower_bound=-500,
                        )

In [7]:
# run fxn
fnames = get_heads(ws=pf.new_d)
print(fnames)

# add head arrays at all kpers as obs
for f in fnames:
    kper = f.split(".")[0].split("kper")[-1]
    obs_df = pf.add_observations(f, 
                        insfile=f+".ins",  
                        prefix=f.split(".")[0],)

#add fxn to forward run
pf.add_py_function(os.path.join("..","useful_fxns.py"),"get_heads()",is_pre_cmd=False)

['hds_layer0_kper0.txt', 'hds_layer0_kper1.txt', 'hds_layer0_kper2.txt', 'hds_layer0_kper3.txt', 'hds_layer0_kper4.txt', 'hds_layer0_kper5.txt', 'hds_layer0_kper6.txt', 'hds_layer0_kper7.txt', 'hds_layer0_kper8.txt', 'hds_layer0_kper9.txt', 'hds_layer0_kper10.txt', 'hds_layer0_kper11.txt', 'hds_layer0_kper12.txt', 'hds_layer0_kper13.txt', 'hds_layer0_kper14.txt', 'hds_layer0_kper15.txt', 'hds_layer0_kper16.txt', 'hds_layer0_kper17.txt', 'hds_layer0_kper18.txt', 'hds_layer0_kper19.txt', 'hds_layer0_kper20.txt', 'hds_layer0_kper21.txt', 'hds_layer0_kper22.txt', 'hds_layer0_kper23.txt', 'hds_layer0_kper24.txt']


In [8]:
#---build pst---#
pst = pf.build_pst()

pst.control_data.noptmax = 0
pst.write(os.path.join(pf.new_d,"freyberg.pst"),version=2)
pyemu.os_utils.run("pestpp-glm freyberg.pst",cwd=pf.new_d)
pst = pyemu.Pst(os.path.join(pf.new_d,"freyberg.pst"))
assert pst.phi < 1e-7

noptmax:0, npar_adj:175, nnz_obs:20000
noptmax:0, npar_adj:175, nnz_obs:20000


             pestpp-glm: a tool for GLM parameter estimation and FOSM uncertainty analysis

                                   by The PEST++ Development Team


version: 5.2.6
binary compiled on Aug 15 2023 at 16:48:53

started at 10/16/24 15:16:21
...processing command line: ' ./pestpp-glm freyberg.pst'
...using serial run manager

using control file: "freyberg.pst"

in directory: "/Users/katherinemarkovich/Desktop/GMDSI_notebooks/tutorials/part2_11_rme/tmp_pst"
on host: "katie-mac"

processing control file freyberg.pst

Note: 'NOPTMAX' == 0, switching to forgiveness mode when checking inputs

noptmax = 0, resetting max_run_fail = 1
parameter warning: 11 parameters are at upper bound
14 parameter warnings, see rec file 
checking model IO files...done
              starting serial run manager ...






    ---  starting serial run manager for 1 runs ---    


10/16/24 15:16:21 processing template files with 

Okay, we've got our simple `pestpp-glm` run built and ready to go, now it's time to generate the response matrix! We do this simply by running GLM to calculate 2-point derivates (aka differencing an increment with the base run). But one critical detail here is establishing what that increment value should be. 

For calculating a Jacobian, the increment value must satisfy the derivative assumption by being as small as possible to approximate the derivate while also being large enough to produce a response in the outputs. This is where the reponse matrix deviates from a Jacobian, because it is not beholden to the derivative assumption. Instead, we want to use a larger increment to ensure we get a large response in the outputs. #internal note, it might be cool to demonstrate this

In [13]:
pst = pyemu.Pst(os.path.join(pf.new_d,"freyberg.pst"))
pst.control_data.noptmax = -1

#increment values are set by par groups (not by indiv pars)
par_gp = pst.parameter_groups
derinc_dict = {
    'welrate':{
                "derinc":10,
               "inctyp":"absolute"
               },
           }
for k,v in derinc_dict.items():
    par_gp.loc[k,'derinc'] = derinc_dict[k]["derinc"]
    par_gp.loc[k,'inctyp'] = derinc_dict[k]["inctyp"]


pst.write(os.path.join(pf.new_d,"freyberg.pst"),version=2)
# run
pyemu.os_utils.start_workers(pf.new_d,  # the folder which contains the "template" PEST dataset
                            'pestpp-glm',  # the PEST software version we want to run
                            "freyberg.pst",  # the control file to use with PEST
                            num_workers=4,  # how many agents to deploy
                            worker_root=".",# where to deploy the agent directories; relative to where python is running
                            master_dir="freyberg_rm",  # the manager directory
                            cleanup=True,
                            )

noptmax:-1, npar_adj:175, nnz_obs:20000


             pestpp-glm: a tool for GLM parameter estimation and FOSM uncertainty analysis

                                   by The PEST++ Development Team


version: 5.2.6
binary compiled on Aug 15 2023 at 16:48:53

started at 10/16/24 15:57:19
...processing command line: ' ./pestpp-glm freyberg.pst /h :4004'
...using panther run manager in master mode using port 4004

using control file: "freyberg.pst"

in directory: "/Users/katherinemarkovich/Desktop/GMDSI_notebooks/tutorials/part2_11_rme/freyberg_rm"
on host: "katie-mac"

processing control file freyberg.pst
parameter warning: 11 parameters are at upper bound
parameter error: PNAME:WEL_11_INST:0_PTYPE:GR_USECOL:3_PSTYLE:D_IDX0:0_IDX1:24 initial value is less than lower bound
parameter error: PNAME:WEL_12_INST:0_PTYPE:GR_USECOL:3_PSTYLE:D_IDX0:0_IDX1:24 initial value is less than lower bound
parameter error: PNAME:WEL_13_INST:0_PTYPE:GR_USECOL:3_PSTYLE:D_IDX0:0_IDX1:24 initial value is les

Exception: start_workers() master returned non-zero: 1